In [1]:
import pandas as pd
import numpy as np
import datetime as dt
from pathlib import Path
from os import chdir
import os.path

In [2]:
# import Excel sheets # case-sensitive
path = r'C:\Users\Jacob Shughrue\Dropbox\Coding\Python\Harrys\harrys_project'
chdir(path)  # set working directory

# import Excel sheets # case-sensitive
filename = 'exercise_data.xlsx'
sheet1 = 'data'
sheet2 = 'Viewable Products'
data = pd.read_excel(filename, sheet_name=sheet1)
products = pd.read_excel(filename, sheet_name=sheet2)

In [3]:
data.head(5)

,id,viewable_product_id,created_at,quantity,user_id,removed_at,created_by_client_type,removed_by_client_type
0,1493735,511,2016-07-28 12:34:42.906,1,36867,NaT,www,NaN
1,1607724,490,2016-08-28 19:00:06.840,1,42060,NaT,www,NaN
2,1539095,511,2016-08-10 20:02:59.016,1,52992,2016-12-05 21:31:19.158,www,www
3,1515988,511,2016-08-04 12:47:41.972,1,65654,2016-09-12 11:20:09.225,www,www
4,1495173,511,2016-07-28 19:49:50.478,1,95304,NaT,www,NaN


In [4]:
products.head(5)

,abbrev,viewable_product_id,price,starter_set_count,other_set_count,blade_count,handle_count,shave_gel_count,shave_cream_count,face_wash_count,aftershave_count,lipbalm_count,razorstand_count,face_lotion_count,travel_kit_count
0,ShaveCream,110,8,0,0,0,0,0,1,0,0,0,0,0,0
1,ShaveCream,120,12,0,0,0,0,0,1,0,0,0,0,0,0
2,DailyFaceWash,318,7,0,0,0,0,0,0,1,0,0,0,0,0
3,DailyFaceLotionSPF15,446,8,0,0,0,0,0,0,0,0,0,0,1,0
4,HarrysBlades,488,8,0,0,4,0,0,0,0,0,0,0,0,0


In [5]:
# join two csv files on "viewable_product_id"
df = pd.merge(data, products)
df.head(5)

,id,viewable_product_id,created_at,quantity,user_id,removed_at,created_by_client_type,removed_by_client_type,abbrev,price,...,blade_count,handle_count,shave_gel_count,shave_cream_count,face_wash_count,aftershave_count,lipbalm_count,razorstand_count,face_lotion_count,travel_kit_count
0,1493735,511,2016-07-28 12:34:42.906,1,36867,NaT,www,NaN,BladesPlan,15,...,8,0,0,0,0,0,0,0,0,0
1,1539095,511,2016-08-10 20:02:59.016,1,52992,2016-12-05 21:31:19.158,www,www,BladesPlan,15,...,8,0,0,0,0,0,0,0,0,0
2,1515988,511,2016-08-04 12:47:41.972,1,65654,2016-09-12 11:20:09.225,www,www,BladesPlan,15,...,8,0,0,0,0,0,0,0,0,0
3,1495173,511,2016-07-28 19:49:50.478,1,95304,NaT,www,NaN,BladesPlan,15,...,8,0,0,0,0,0,0,0,0,0
4,1541667,511,2016-08-11 15:33:30.482,1,182104,NaT,www,NaN,BladesPlan,15,...,8,0,0,0,0,0,0,0,0,0


In [6]:
# rename columns for simplicity 
df = df.rename(columns={
    "created_at": "plan_start_date",
    "removed_at": "plan_end_date",
    "created_by_client_type": "plan_start_type",
    "removed_by_client_type": "plan_end_type",
    "abbrev": "product_name",
    "viewable_product_id": "product_id"
})

In [7]:
# replace subscription aquisition type of "www" with "web"
df['plan_start_type'] = df['plan_start_type'].str.replace('www', 'web')
df['plan_end_type'] = df['plan_end_type'].str.replace('www', 'web')

In [8]:
# replace a blank subscription end date wtih "on_going" to indicate the subscription is still active
df['plan_end_type'] = df['plan_end_type'].replace(np.nan, "on_going")
df['plan_end_date'] = df['plan_end_date'].replace(np.nan, "on_going")


In [9]:
# create a variable equivalent to the last date in file, this will be used to find deltas
feb012017 = date_time_obj = dt.datetime.strptime('2017-02-01 14:24:00', '%Y-%m-%d %H:%M:%S')

# replace subscription termination value of 'on_going' with last date in file to allow for calculations
df.loc[df.plan_end_date == 'on_going', 'plan_end_date'] = feb012017

In [10]:
# drop rows with data quality issues:
dq_rule_failure = df.loc[
    (df['plan_end_type'] == 'on_going') & df['plan_end_date'].notnull() & (df['plan_end_date'] != feb012017)]

df = df[~df['user_id'].isin(dq_rule_failure['user_id'])]
print("{} rows dropped due to data quality issues".format(dq_rule_failure['user_id'].count()))

77 rows dropped due to data quality issues


In [11]:
# preventative data quality rule 
# dropping rows like the below because a subscription cannot be both on_going and have an 'end_date'
dq_rule_failure.iloc[0:5,np.r_[5,7]]

,plan_end_date,plan_end_type
145,2016-11-23 16:30:15.713000,on_going
557,2016-08-02 18:46:07.573000,on_going
588,2016-11-15 22:20:20.320000,on_going
651,2017-01-12 19:07:38.111000,on_going
815,2016-08-15 17:01:23.116000,on_going


In [12]:
# change type of column to datetime
df['plan_end_date'] = pd.to_datetime(df['plan_end_date'], format='%Y-%m-%d %H:%M:%S.%f')

# create columns for the first customer sale date and last customer sale date for the entire customer lifetime
df['first_cust_sale_date'] = (df.groupby(['user_id'])['plan_start_date'].transform('min'))

df['last_cust_sale_date'] = (df.groupby(['user_id'])['plan_start_date'].transform('max'))

In [13]:
df.iloc[5:10,np.r_[2,4:9,22:24]]

,plan_start_date,user_id,plan_end_date,plan_start_type,plan_end_type,product_name,first_cust_sale_date,last_cust_sale_date
5,2016-08-21 17:31:40.039,224478,2016-11-14 03:21:00.058,web,web,BladesPlan,2016-08-13 03:39:29.100,2016-11-15 01:40:42.845
6,2016-11-14 03:21:00.047,224478,2016-11-15 01:40:42.870,web,web,BladesPlan,2016-08-13 03:39:29.100,2016-11-15 01:40:42.845
7,2016-11-15 01:40:42.845,224478,2017-02-01 14:24:00.000,web,on_going,BladesPlan,2016-08-13 03:39:29.100,2016-11-15 01:40:42.845
8,2016-07-24 18:40:30.242,232261,2017-01-05 12:26:04.896,web,web,BladesPlan,2016-07-24 18:40:30.242,2016-07-24 18:40:30.242
9,2016-08-08 19:26:14.265,233681,2017-02-01 14:24:00.000,web,on_going,BladesPlan,2016-08-08 19:26:14.265,2016-08-08 19:26:14.265


In [14]:
# create temporary column: get the maximum plan_end_date per customer less the first_cust_sale_date column
df['max_plan_end_date_per_user'] = round(
    (df.groupby(['user_id'])['plan_end_date'].transform('max') - df['first_cust_sale_date']).dt.total_seconds() / 86400,
    1)  # where 86400 seconds is one day

In [15]:
# create a column,total_cust_lifetime_length_days, counting how long the customer has been with the company
df['total_custtime_life_length_days'] = df['max_plan_end_date_per_user']  # assign default values for column

row_is_active = df["plan_end_type"] == "on_going"  # assign variable for readability

df.loc[row_is_active, 'total_cust_lifetime_length_days'] = ((feb012017 -
                                                df[row_is_active][
                                                'first_cust_sale_date']).dt.total_seconds() / 86400).round(1)

In [16]:
# create a column indicating if the customer has returned to purchase additional items
df['repeat_cust_flag'] = (df['last_cust_sale_date'] - df[
    'first_cust_sale_date']).dt.total_seconds() / 86400 >= 1

In [17]:
df.iloc[:5,np.r_[0,22:27]]

,id,first_cust_sale_date,last_cust_sale_date,max_plan_end_date_per_user,total_custtime_life_length_days,total_cust_lifetime_length_days
0,1493735,2016-07-28 12:34:42.906,2016-07-28 12:34:42.906,188.1,188.1,188.1
1,1539095,2016-08-10 20:02:59.016,2016-08-10 20:02:59.016,117.1,117.1,NaN
2,1515988,2016-08-04 12:47:41.972,2016-08-04 12:47:41.972,38.9,38.9,NaN
3,1495173,2016-07-28 19:49:50.478,2016-07-28 19:49:50.478,187.8,187.8,187.8
4,1541667,2016-08-11 15:33:30.482,2016-08-11 15:33:30.482,174.0,174.0,174.0


In [18]:
# create a column for line subscription duration days - indicates how long a customer subscribed to this line item
row_is_active = df["plan_end_type"] == "on_going"  # assign variable for readability


df['line_subscription_duration_days'] = np.where(
    row_is_active, 
    (((feb012017 - df['plan_start_date']).dt.total_seconds()) / 86400).round(1), 
    (((df['plan_end_date'] - df['plan_start_date']).dt.total_seconds()) / 86400).round(1))

In [19]:
# creation of line_one_time_purchase_flag - if "subscriped" for less then half a day, it was a one time purchase
df['line_one_time_purchase_flag'] = df['line_subscription_duration_days'] < 0.5

In [20]:
# create a count of how many times a customber started a new product subscription
df['subscription_occurrence_count'] = df.groupby(['user_id'])['user_id'].transform('count')

In [21]:
# create cancelled_line_flag
df['cancelled_line_flag'] = np.where(
    df['line_one_time_purchase_flag'],  # if: value is "one_time_purchase" then, TRUE
    "one_time_purchase",
    df['plan_end_type'] != 'on_going'  # else: if value is not "on_going" then TRUE
)

In [22]:
df.iloc[5:10,np.r_[0,28:32]]

,id,line_subscription_duration_days,line_one_time_purchase_flag,subscription_occurrence_count,cancelled_line_flag
5,1576048,84.4,False,6,True
6,1841486,0.9,False,6,True
7,1844966,78.5,False,6,False
8,1480591,164.7,False,1,True
9,1531559,176.8,False,1,False


In [23]:
# create cust_cancellation_count column
# counts the unique number of times a user cancelled a product (aka cancelled_line_flag =='True')
# then uses a merge to apply the distinct count to the df
df = df.merge(
    df.loc[df['cancelled_line_flag'].astype(str) == 'True', 'user_id'].value_counts().rename(
        'cust_cancellation_count'),
    how='outer', left_on='user_id', right_index=True).fillna(
    0)  # uses a full outer join so customers that have not cancelled are not dropped

df['cust_cancellation_count'] = pd.to_numeric(df['cust_cancellation_count'], downcast="integer") # assign as an integer

In [24]:
# create a column for count_of_unique_lifetime_products
# df['count_of_unique_lifetime_products'] = df.groupby('user_id')['product_name'].nunique() # stand alone fucntion before merge
df = df.merge(df.groupby('user_id')['product_name'].nunique().rename('count_of_unique_lifetime_products'),
              how='outer',
              left_on='user_id', right_index=True)


In [25]:
# create a lookup table for a 'Product Type' market lens
product_type_table = pd.DataFrame(
    {'product_name': [
        'ShaveCream', 'DailyFaceWash', 'DailyFaceLotionSPF15', 'HarrysBlades', 'Blades2GelsPlan',
        'Blades,Gel,PostShave', 'FoamingShaveGel',
        'PostShaveBalm', 'BladesPlan', 'RazorBlades', 'Blades1GelPlan', 'ShaveGroomPlan'],
        'product_type': ['Pre/Post Shave', 'FaceCare', 'FaceCare', 'Blades', 'Bundle',
                         'Bundle', 'Pre/Post Shave', 'Pre/Post Shave', 'Blades', 'Blades', 'Bundle', 'Bundle']})

In [26]:
product_type_table

,product_name,product_type
0,ShaveCream,Pre/Post Shave
1,DailyFaceWash,FaceCare
2,DailyFaceLotionSPF15,FaceCare
3,HarrysBlades,Blades
4,Blades2GelsPlan,Bundle
5,"Blades,Gel,PostShave",Bundle
6,FoamingShaveGel,Pre/Post Shave
7,PostShaveBalm,Pre/Post Shave
8,BladesPlan,Blades
9,RazorBlades,Blades


In [27]:
# create a column for product_type
df = df.merge(product_type_table, how='left', on='product_name')

In [28]:
# create a column for the count of sales lines per customer
df['cust_id_occurrence_count'] = df.groupby(['user_id'])['user_id'].transform('count')

In [29]:
# create column for customer_subscription_change_flag
# if the customer is a repeat customer, has cancelled a subscription and has had more then one product then TRUE
df['customer_subscription_change_flag'] = (df['repeat_cust_flag'] == True) & (df['cust_cancellation_count'] >= 1) & (
        df['count_of_unique_lifetime_products'] > 1)

In [30]:
# set the 'id' column to be the index column
df.set_index("id", inplace=True)

In [31]:
df = df.drop(columns=['max_plan_end_date_per_user'])  # drop temporary column

In [32]:
# reorder columns for export
df = df[[
    'cust_id_occurrence_count',
    'cancelled_line_flag',
    'cust_cancellation_count',
    'line_subscription_duration_days',
    'line_one_time_purchase_flag',
    'first_cust_sale_date',
    'last_cust_sale_date',
    'repeat_cust_flag',
    'count_of_unique_lifetime_products',
    # 'first_subscription_flag', # not including this
    'subscription_occurrence_count',
    'total_cust_lifetime_length_days',
    'customer_subscription_change_flag',
    'product_id',
    'user_id',
    'plan_start_type',
    'plan_end_type',
    'plan_start_date',
    'plan_end_date',
    'product_name',
    'product_type',
    'quantity',
    'price',
    'starter_set_count',
    'other_set_count',
    'blade_count',
    'handle_count',
    'shave_gel_count',
    'shave_cream_count',
    'face_wash_count',
    'aftershave_count',
    'lipbalm_count',
    'razorstand_count',
    'face_lotion_count',
    'travel_kit_count']]

In [33]:
# sort
df = df.sort_values(['user_id', 'id'], ascending=[True, True])

In [34]:
df.iloc[:10,np.r_[:20]]

,cust_id_occurrence_count,cancelled_line_flag,cust_cancellation_count,line_subscription_duration_days,line_one_time_purchase_flag,first_cust_sale_date,last_cust_sale_date,repeat_cust_flag,count_of_unique_lifetime_products,subscription_occurrence_count,total_cust_lifetime_length_days,customer_subscription_change_flag,product_id,user_id,plan_start_type,plan_end_type,plan_start_date,plan_end_date,product_name,product_type
id,,,,,,,,,,,,,,,,,,,,
1507701,1,False,0,183.7,False,2016-08-01 20:52:35.770,2016-08-01 20:52:35.770,False,1,1,183.7,False,511,968,web,on_going,2016-08-01 20:52:35.770,2017-02-01 14:24:00.000,BladesPlan,Blades
1526753,1,False,0,177.8,False,2016-08-07 18:25:11.394,2016-08-07 18:25:11.394,False,1,1,177.8,False,504,5404,web,on_going,2016-08-07 18:25:11.394,2017-02-01 14:24:00.000,Blades2GelsPlan,Bundle
1489315,1,True,1,104.7,False,2016-07-26 22:07:14.249,2016-07-26 22:07:14.249,False,1,1,0.0,False,511,7938,web,web,2016-07-26 22:07:14.249,2016-11-08 14:51:27.497,BladesPlan,Blades
1526391,5,True,1,165.3,False,2016-08-07 17:10:30.203,2017-01-20 01:22:21.351,True,2,5,0.0,True,511,17951,web,web,2016-08-07 17:10:30.203,2017-01-20 01:16:13.994,BladesPlan,Blades
2050396,5,one_time_purchase,1,0.0,True,2016-08-07 17:10:30.203,2017-01-20 01:22:21.351,True,2,5,0.0,True,110,17951,web,web,2017-01-20 01:16:13.981,2017-01-20 01:22:12.134,ShaveCream,Pre/Post Shave
2050416,5,one_time_purchase,1,0.0,True,2016-08-07 17:10:30.203,2017-01-20 01:22:21.351,True,2,5,0.0,True,110,17951,web,web,2017-01-20 01:22:12.116,2017-01-20 01:22:21.367,ShaveCream,Pre/Post Shave
2050417,5,one_time_purchase,1,0.0,True,2016-08-07 17:10:30.203,2017-01-20 01:22:21.351,True,2,5,0.0,True,511,17951,web,web,2017-01-20 01:22:12.123,2017-01-20 01:22:21.384,BladesPlan,Blades
2050418,5,False,1,12.5,False,2016-08-07 17:10:30.203,2017-01-20 01:22:21.351,True,2,5,177.9,True,110,17951,web,on_going,2017-01-20 01:22:21.351,2017-02-01 14:24:00.000,ShaveCream,Pre/Post Shave
1447565,2,False,0,193.8,False,2016-07-22 19:54:58.800,2016-07-25 00:33:29.301,True,1,2,193.8,False,511,19323,system,on_going,2016-07-22 19:54:58.800,2017-02-01 14:24:00.000,BladesPlan,Blades


In [35]:
# write to csv
#df.to_csv(os.path.join(r'C:\Users\Jacob Shughrue\Dropbox\Coding\Python\Harrys', 'df_python_export.csv'))